# MLFlow Workshop - Sequence 4 - Model Registry API

## Install dependencies

In [1]:
!pip -q install mlflow boto3 pyngrok

     |████████████████████████████████| 17.8 MB 407 kB/s 
     |████████████████████████████████| 132 kB 49.0 MB/s 
     |████████████████████████████████| 745 kB 44.5 MB/s 
     |████████████████████████████████| 209 kB 64.2 MB/s 
     |████████████████████████████████| 146 kB 69.6 MB/s 
     |████████████████████████████████| 596 kB 60.0 MB/s 
     |████████████████████████████████| 181 kB 55.7 MB/s 
     |████████████████████████████████| 81 kB 9.0 MB/s 
     |████████████████████████████████| 79 kB 8.2 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 8.9 MB 43.5 MB/s 
     |████████████████████████████████| 79 kB 7.4 MB/s 
     |████████████████████████████████| 127 kB 54.2 MB/s 
     |████████████████████████████████| 78 kB 6.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the

## Check and see if SQL is installed

In [2]:
#conda install -c conda-forge ipython-sql #must install to kernel if running locally
%load_ext sql

In [3]:
from pyngrok import ngrok
from getpass import getpass

In [4]:
def start_tunnel():
  # Terminate open tunnels if exist
  ngrok.kill()
  # Get your authtoken from https://dashboard.ngrok.com/auth
  NGROK_AUTH_TOKEN = getpass('Enter the ngrok authtoken: ')
  ngrok.set_auth_token(NGROK_AUTH_TOKEN)
  # Open an HTTPs tunnel on port 5000 for http://localhost:5000
  ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
  print("MLflow Tracking UI:", ngrok_tunnel.public_url)

## Using an SQLite server for tracking (locally)

In [5]:
!touch mlflow.sqlite #this will only persist for the duration of the Colab session unless you move it to your Google Drive

In [35]:
%env MLFLOW_TRACKING_URI=http://localhost:5000

env: MLFLOW_TRACKING_URI=http://localhost:5000


In [41]:
get_ipython().system_raw(
    "mlflow server --backend-store-uri sqlite:///mlflow.sqlite --host 0.0.0 --port 5000 --default-artifact-root mlruns &"
    )

# get_ipython().system_raw(
#     "mlflow server --backend-store-uri sqlite:///mlflow.sqlite --host 0.0.0 --port 5000 --serve-artifacts --artifacts-destination /content/mlruns &"
#     )

In [42]:
!mlflow run --env-manager=local https://github.com/mlflow/mlflow-example.git -P alpha=0.2

t=2022-06-20T14:29:11+0000 lvl=warn msg="failed to open private leg" id=6f4baef168fc privaddr=localhost:5000 err="dial tcp 127.0.0.1:5000: connect: connection refused"
t=2022-06-20T14:29:11+0000 lvl=warn msg="failed to open private leg" id=abe4ea0e06f3 privaddr=localhost:5000 err="dial tcp 127.0.0.1:5000: connect: connection refused"
t=2022-06-20T14:29:12+0000 lvl=warn msg="failed to open private leg" id=fafb92565fda privaddr=localhost:5000 err="dial tcp 127.0.0.1:5000: connect: connection refused"
t=2022-06-20T14:29:13+0000 lvl=warn msg="failed to open private leg" id=6f620a1d4dea privaddr=localhost:5000 err="dial tcp 127.0.0.1:5000: connect: connection refused"


2022/06/20 14:29:13 INFO mlflow.projects.utils: === Fetching project from https://github.com/mlflow/mlflow-example.git into /tmp/tmpnawuvzdx ===
2022/06/20 14:29:18 INFO mlflow.projects.utils: === Created directory /tmp/tmp8ky3uhb3 for downloading remote URIs passed to arguments of type 'path' ===
2022/06/20 14:29:18 INFO mlflow.projects.backend.local: === Running command 'python train.py 0.2 0.1' in run with ID '9090f2c4a3e740aaac4c7e145d70969e' === 
Elasticnet model (alpha=0.200000, l1_ratio=0.100000):
  RMSE: 0.7836984021909766
  MAE: 0.6142020452688988
  R2: 0.20673590971167466
2022/06/20 14:29:23 INFO mlflow.projects: === Run (ID '9090f2c4a3e740aaac4c7e145d70969e') succeeded ===


In [38]:
start_tunnel()

Enter the ngrok authtoken: ··········
MLflow Tracking UI: https://9abc-35-185-229-27.ngrok.io


In [40]:
# To kill MLFlow Server instances running in the background, uncomment & run:
!pkill -f gunicorn

In [10]:
!ls mlruns

## Using an SQL tracking server & S3 bucket for artifact storage

In [ ]:
%env AWS_ACCESS_KEY_ID=jwhttmo4qhgfncxxdftqlwpxfaia
%env AWS_SECRET_ACCESS_KEY=jyhyd7bxmbax3w3mzfi55tscfk5t34rotzj3nbg2e4nwpofxwuh2y

%env MLFLOW_S3_ENDPOINT_URL=https://gateway.eu1.storjshare.io/

env: AWS_ACCESS_KEY_ID=jwhttmo4qhgfncxxdftqlwpxfaia
env: AWS_SECRET_ACCESS_KEY=jyhyd7bxmbax3w3mzfi55tscfk5t34rotzj3nbg2e4nwpofxwuh2y
env: MLFLOW_S3_ENDPOINT_URL=https://gateway.eu1.storjshare.io/


In [ ]:
!mlflow server --backend-store-uri sqlite:///mlflow.sqlite --default-artifact-root s3://demobucket --gunicorn-opts "--log-level debug"

[2022-06-19 18:26:59 +0000] [319] [DEBUG] Current configuration:
  config: ./gunicorn.conf.py
  wsgi_app: None
  bind: ['127.0.0.1:5000']
  backlog: 2048
  workers: 4
  worker_class: sync
  threads: 1
  worker_connections: 1000
  max_requests: 0
  max_requests_jitter: 0
  timeout: 30
  graceful_timeout: 30
  keepalive: 2
  limit_request_line: 4094
  limit_request_fields: 100
  limit_request_field_size: 8190
  reload: False
  reload_engine: auto
  reload_extra_files: []
  spew: False
  check_config: False
  print_config: False
  preload_app: False
  sendfile: None
  reuse_port: False
  chdir: /content
  daemon: False
  raw_env: []
  pidfile: None
  worker_tmp_dir: None
  user: 0
  group: 0
  umask: 0
  initgroups: False
  tmp_upload_dir: None
  secure_scheme_headers: {'X-FORWARDED-PROTOCOL': 'ssl', 'X-FORWARDED-PROTO': 'https', 'X-FORWARDED-SSL': 'on'}
  forwarded_allow_ips: ['127.0.0.1']
  accesslog: None
  disable_redirect_access_to_syslog: False
  access_log_format: %(h)s %(l)s %(u)s

## Servable

In [ ]:
!mlflow server --backend-store-uri sqlite:///mlflow.sqlite --default-artifact-root mlflow-artifacts --artifacts-destination s3://demobucket --serve-artifacts

[2022-06-19 19:41:03 +0000] [778] [INFO] Starting gunicorn 20.1.0
[2022-06-19 19:41:03 +0000] [778] [INFO] Listening at: http://127.0.0.1:5000 (778)
[2022-06-19 19:41:03 +0000] [778] [INFO] Using worker: sync
[2022-06-19 19:41:03 +0000] [781] [INFO] Booting worker with pid: 781
[2022-06-19 19:41:03 +0000] [782] [INFO] Booting worker with pid: 782
[2022-06-19 19:41:03 +0000] [783] [INFO] Booting worker with pid: 783
[2022-06-19 19:41:03 +0000] [784] [INFO] Booting worker with pid: 784
[2022-06-19 20:32:20 +0000] [778] [INFO] Handling signal: int
[2022-06-19 20:32:20 +0000] [781] [INFO] Worker exiting (pid: 781)
[2022-06-19 20:32:20 +0000] [783] [INFO] Worker exiting (pid: 783)
[2022-06-19 20:32:20 +0000] [782] [INFO] Worker exiting (pid: 782)
[2022-06-19 20:32:20 +0000] [784] [INFO] Worker exiting (pid: 784)

Aborted!
[2022-06-19 20:32:21 +0000] [778] [INFO] Shutting down: Master


In [ ]:
import boto3

In [ ]:
client = boto3.client('s3', endpoint_url='https://gateway.storjshare.io/')

In [ ]:
client.list_buckets()

{'Buckets': [{'CreationDate': datetime.datetime(2022, 6, 19, 14, 50, 50, 457000, tzinfo=tzlocal()),
   'Name': 'demobucket'}],
 'Owner': {'DisplayName': 'minio',
  'ID': '02d6176db174dc93cb1b899f7c6078f08654445fe8cf1b6ce98d8855f66bdbf4'},
 'ResponseMetadata': {'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '368',
   'content-security-policy': 'block-all-mixed-content',
   'content-type': 'application/xml',
   'date': 'Sun, 19 Jun 2022 16:58:56 GMT',
   'server': 'MinIO',
   'vary': 'Origin',
   'x-amz-request-id': '16FA141EE242BAF7',
   'x-xss-protection': '1; mode=block'},
  'HTTPStatusCode': 200,
  'HostId': '',
  'RequestId': '16FA141EE242BAF7',
  'RetryAttempts': 0}}

In [ ]:
client.upload_file("mlflow.sqlite", "demobucket", 'jvvt5omvvtp5gj67mxfkj644l7ma')

In [ ]:
client.list_objects(Bucket='demobucket')

{'Contents': [{'ETag': '"849e1101f85879ea954301b60efee0bf"',
   'Key': 'jvvt5omvvtp5gj67mxfkj644l7ma',
   'LastModified': datetime.datetime(2022, 6, 19, 17, 0, 17, 298000, tzinfo=tzlocal()),
   'Owner': {'DisplayName': 'minio',
    'ID': '02d6176db174dc93cb1b899f7c6078f08654445fe8cf1b6ce98d8855f66bdbf4'},
   'Size': 126976,
   'StorageClass': 'STANDARD'}],
 'Delimiter': '',
 'EncodingType': 'url',
 'IsTruncated': False,
 'Marker': '',
 'MaxKeys': 1000,
 'Name': 'demobucket',
 'Prefix': '',
 'ResponseMetadata': {'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '635',
   'content-security-policy': 'block-all-mixed-content',
   'content-type': 'application/xml',
   'date': 'Sun, 19 Jun 2022 17:01:02 GMT',
   'server': 'MinIO',
   'vary': 'Origin',
   'x-amz-request-id': '16FA143C4ABDE407',
   'x-xss-protection': '1; mode=block'},
  'HTTPStatusCode': 200,
  'HostId': '',
  'RequestId': '16FA143C4ABDE407',
  'RetryAttempts': 0}}

In [ ]:
client.download_file("demobucket", 'jvvt5omvvtp5gj67mxfkj644l7ma',"mlflow.sqlite")